# Basic Usage

In order to improve your Nengo model, the most basic thing you can try is to simply insert the following code at the top of your file:

**```
import nengolib; nengolib.patch()
```**

This will automatically improve its:
 - encoders
 - spiking LIF model (prior to `Nengo 2.1.1`)
 - evaluation points
 - decoders
 
Usually this only makes a difference for difficult representations involving small numbers of neurons and/or high dimensionality.
 
See `doc/notebooks/examples/network.ipynb` and `doc/notebooks/examples/connection.ipynb` for details.
 
These benefits are demonstrated below by building a communication channel for a vector.

In [ ]:
%pylab inline
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

import numpy as np

import nengo
from nengo.utils.numpy import rmse

In [ ]:
def trial(n=200, d=16, T=0.5):
    with nengo.Network() as model:
        stim = nengo.Node(output=lambda x: np.squeeze(
            nengo.dists.UniformHypersphere(surface=True).sample(d, 1)))
        
        x = nengo.Ensemble(n, d, neuron_type=nengo.neurons.LIFRate())
        output = nengo.Node(size_in=d)
                                 
        nengo.Connection(stim, x, synapse=None)
        nengo.Connection(x, output, synapse=None)
        
        p_stim = nengo.Probe(stim)
        p_output = nengo.Probe(output)
        
    with nengo.Simulator(model) as sim:
        sim.run(T, progress_bar=False)
        
    return np.mean(rmse(sim.data[p_stim], sim.data[p_output], axis=1))

In [ ]:
def experiment(num_trials=100, *args, **kwargs):
    trials = []
    for _ in range(num_trials):
        trials.append(trial(*args, **kwargs))
        
    return trials

default = experiment()

This experiment returns the RMSE from a number of random trials.

Now we import nengolib, monkey-patch the classes, and re-run the experiment. 

In [ ]:
import nengolib; nengolib.patch()

improved = experiment()

nengolib.unpatch()  # reset back to normal

In [ ]:
pylab.figure()
pylab.title("Comparison of Models (%.1f%%)" % (100 * np.mean(default) / np.mean(improved)))
pylab.boxplot([default, improved])
pylab.xticks([1, 2], ['Default', 'Improved'])
pylab.ylabel("RMSE")
pylab.show()

We see the there is a slight improvement in the mean (roughly 1-2%).

These differences become negligible / invisible as the number of neurons is increased relative to the dimensionality, and depending on the demands of the particular model -- so no promises, but it's worth the try!